In [2]:
from google.colab import drive
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/Colab_Notebooks2/LLM_files'

Mounted at /content/drive


In [ ]:
# Google Colab Setup Script for LLM + FA Detection EDA Flow

# 1. System update & dependencies
!apt-get update -qq && apt-get install -y build-essential clang bison flex libreadline-dev \
  gawk tcl-dev libffi-dev git mercurial graphviz xdot pkg-config python3 python3-pip \
  libboost-system-dev libboost-python-dev libboost-filesystem-dev zlib1g-dev

# 2. Clone and build Yosys
!git clone https://github.com/YosysHQ/yosys.git
%cd yosys
!git submodule update --init --recursive
!make -j$(nproc)
!make install
!cp -r /content/yosys SAVE_DIR



In [ ]:
!cp -r /content/yosys /content/drive/MyDrive/Colab_Notebooks2/LLM_files/

In [161]:
# 3. Go back and create project directories
import os
%cd /
verilog_path = os.path.join(SAVE_DIR, 'llm_fa/verilog_src')
!mkdir -p /content/llm_fa/verilog_src /content/llm_fa/scripts /content/llm_fa/stats /content/llm_fa/llm

# 4. Create example FA Verilog
fa_verilog = '''
module mystery_adder (
    input  wire [3:0] a,
    input  wire [3:0] b,
    input  wire       cin,
    output wire [3:0] sum,
    output wire       cout
);
    wire c0, c1, c2, c3;
    wire n00, n01, n02, n03, n04, n05, n06, n07;
    assign n00 = a[0] ^ b[0];
    assign n01 = n00 ^ cin;
    assign n02 = a[0] & b[0];
    assign n03 = b[0] & cin;
    assign n04 = a[0] & cin;
    assign n05 = n02 | n03;
    assign n06 = n05 | n04;
    assign sum[0] = n01;
    assign c0 = n06;
    wire n10, n11, n12, n13, n14, n15, n16, n17;
    assign n10 = a[1] ^ b[1];
    assign n11 = n10 ^ c0;
    assign n12 = a[1] & b[1];
    assign n13 = b[1] & c0;
    assign n14 = a[1] & c0;
    assign n15 = n12 | n13;
    assign n16 = n15 | n14;
    assign sum[1] = n11;
    assign c1 = n16;
    wire n20, n21, n22, n23, n24, n25, n26, n27;
    assign n20 = a[2] ^ b[2];
    assign n21 = n20 ^ c1;
    assign n22 = a[2] & b[2];
    assign n23 = b[2] & c1;
    assign n24 = a[2] & c1;
    assign n25 = n22 | n23;
    assign n26 = n25 | n24;
    assign sum[2] = n21;
    assign c2 = n26;
    wire n30, n31, n32, n33, n34, n35, n36, n37;
    assign n30 = a[3] ^ b[3];
    assign n31 = n30 ^ c2;
    assign n32 = a[3] & b[3];
    assign n33 = b[3] & c2;
    assign n34 = a[3] & c2;
    assign n35 = n32 | n33;
    assign n36 = n35 | n34;
    assign sum[3] = n31;
    assign cout = n36;

endmodule
'''

with open("/content/llm_fa/verilog_src/fa_example_hard.v", "w") as f:
    f.write(fa_verilog)

# 5. Create FA cell primitive
fa_cell = '''
module top(input a, b, cin, output sum, output cout);
    FA_CELL fa (
        .a(a),
        .b(b),
        .cin(cin),
        .sum(sum),
        .cout(cout)
    );
endmodule

module FA_CELL(
    input  wire a,
    input  wire b,
    input  wire cin,
    output wire sum,
    output wire cout
);
  assign sum  = a ^ b ^ cin;
  assign cout = (a & b) | (cin & (a ^ b));
endmodule
'''
with open("/content/llm_fa/verilog_src/fa_cell.v", "w") as f:
    f.write(fa_cell)

# 6. Create yosys script to generate AIG
synth_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example_hard.v
hierarchy -auto-top
proc; opt; techmap
aigmap
write_aiger -ascii -symbols /content/llm_fa/design.aag
'''
#write_aiger /content/llm_fa/design.aig
with open("/content/llm_fa/scripts/synth.ys", "w") as f:
    f.write(synth_script)

print("✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys")

/
✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys


In [162]:
# 7. Run Yosys synthesis to generate AIG
%cd /content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
!chmod +x yosys
!./yosys -V
!./yosys -q -s /content/llm_fa/scripts/synth.ys


/content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)


In [163]:
# 8-1. Generate LLM prompt to locate FA logic line-by-line in Verilog

prompt_template = f"""

Here is the Verilog source:

{fa_verilog}

For each FA instance in the code:
	•	Add a line-by-line comment at the end of each line that participates in a Full Adder logic, using this format:
wire ab = a & b; // FA #1
wire bc = b & cin; // FA #1
wire ac = a & cin; // FA #1
wire carry = ab | bc | ac; // FA #1
wire sum = a ^ b ^ cin; // FA #1
	•	If a second Full Adder exists in the code, label it as // FA #2, and so on.
	•	Do not change or delete any code.
	•	Leave unrelated lines untouched (no comments).
Please output a complete and compilable Verilog module.

Your output should begin with module and end with endmodule,Please do not include any Markdown code fences like ```verilog in your output. Output only raw Verilog code suitable for saving as a .v file.
"""
with open("/content/llm_fa/llm/llm_prompt_step1.txt", "w") as f:
    f.write(prompt_template)
print("🧠 LLM prompt for step1 generated. Ready to pass to a model like GPT for annotation.")



🧠 LLM prompt for step1 generated. Ready to pass to a model like GPT for annotation.


In [165]:
# 8-2. Call LLM to annotate Verilog code
import os
from openai import OpenAI

# Set your API key
os.environ["OPENAI_API_KEY"] = "USER-API-KEY"
client = OpenAI()

with open("/content/llm_fa/llm/llm_prompt_step1.txt", "r") as f:
    prompt = f.read()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
llm_result_step1 = response.choices[0].message.content
with open("/content/llm_fa/llm/llm_result_step1.txt", "w") as f:
    f.write(llm_result_step1)

print("📝 LLM result saved to llm_result_step1.txt")

module mystery_adder (
    input  wire [3:0] a,
    input  wire [3:0] b,
    input  wire       cin,
    output wire [3:0] sum,
    output wire       cout
);
    wire c0, c1, c2, c3;
    wire n00, n01, n02, n03, n04, n05, n06, n07;
    assign n00 = a[0] ^ b[0]; // FA #1
    assign n01 = n00 ^ cin; // FA #1
    assign n02 = a[0] & b[0]; // FA #1
    assign n03 = b[0] & cin; // FA #1
    assign n04 = a[0] & cin; // FA #1
    assign n05 = n02 | n03; // FA #1
    assign n06 = n05 | n04; // FA #1
    assign sum[0] = n01; // FA #1
    assign c0 = n06; // FA #1
    wire n10, n11, n12, n13, n14, n15, n16, n17;
    assign n10 = a[1] ^ b[1]; // FA #2
    assign n11 = n10 ^ c0; // FA #2
    assign n12 = a[1] & b[1]; // FA #2
    assign n13 = b[1] & c0; // FA #2
    assign n14 = a[1] & c0; // FA #2
    assign n15 = n12 | n13; // FA #2
    assign n16 = n15 | n14; // FA #2
    assign sum[1] = n11; // FA #2
    assign c1 = n16; // FA #2
    wire n20, n21, n22, n23, n24, n25, n26, n27;
    assign n20 =

In [166]:
# 9-1. Convert FA logic to FA cell in Verilog
prompt_template = f"""
You are given a Verilog source code where lines involved in Full Adder logic are marked line-by-line with comments like:
{llm_result_step1}

Your task is to:
1. Detect each group of lines associated with a unique Full Adder based on the // FA #N comments.
2. For each FA group:
	•	Remove all the lines marked with // FA #N
	•	Replace them with a single instantiation of the module fa_cell, using the following format:
FA_CELL fa_N (
  .a(a), .b(b), .cin(cin),
  .sum(sum), .cout(carry)
);
	•	Important: The instance name faN must use the same number N from the comment (e.g., FA #1 → fa1, FA #2 → fa2, etc.)
3. Preserve all unrelated code, including unmarked lines, always keeping the surrounding structure intact.
4. Your output should begin with module and end with endmodule,Please do not include any Markdown code fences like ```verilog in your output. Output only raw Verilog code suitable for saving as a .v file.
"""

with open("/content/llm_fa/llm/llm_prompt_step2.txt", "w") as f:
    f.write(prompt_template)
print("🧠 LLM prompt for step2 generated. Ready to pass to a model like GPT for annotation.")


🧠 LLM prompt for step2 generated. Ready to pass to a model like GPT for annotation.


In [167]:
# 9-2. Call LLM to refactor FA logic to FA cell
import os
from openai import OpenAI

# Set your API key
os.environ["OPENAI_API_KEY"] = "USER-API-KEY"
client = OpenAI()
# 📝 2. Define Verilog with obfuscation (hard to identify FA)

with open("/content/llm_fa/llm/llm_prompt_step2.txt", "r") as f:
    prompt = f.read()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
llm_result_step2 = response.choices[0].message.content
with open("/content/llm_fa/llm/llm_result_step2.txt", "w") as f:
    f.write(llm_result_step2)

with open("/content/llm_fa/verilog_src/fa_example_mod.v", "w") as f:
    f.write(llm_result_step2)


print("📝 LLM result saved to llm_result_step2.txt")

module mystery_adder (
    input  wire [3:0] a,
    input  wire [3:0] b,
    input  wire       cin,
    output wire [3:0] sum,
    output wire       cout
);
    wire c0, c1, c2, c3;
    // FA #1 group replaced
    FA_CELL fa1 (
      .a(a[0]), .b(b[0]), .cin(cin),
      .sum(sum[0]), .cout(c0)
    );
    wire n10, n11, n12, n13, n14, n15, n16, n17;
    // FA #2 group replaced
    FA_CELL fa2 (
      .a(a[1]), .b(b[1]), .cin(c0),
      .sum(sum[1]), .cout(c1)
    );
    wire n20, n21, n22, n23, n24, n25, n26, n27;
    // FA #3 group replaced
    FA_CELL fa3 (
      .a(a[2]), .b(b[2]), .cin(c1),
      .sum(sum[2]), .cout(c2)
    );
    wire n30, n31, n32, n33, n34, n35, n36, n37;
    // FA #4 group replaced
    FA_CELL fa4 (
      .a(a[3]), .b(b[3]), .cin(c2),
      .sum(sum[3]), .cout(cout)
    );

endmodule
📝 LLM result saved to llm_result_step2.txt


In [170]:
# 10. Re-synthesize modified Verilog
synth_mod_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -auto-top
proc; opt; techmap
stat
write_verilog /content/llm_fa/verilog_src/netlist_mod.v
'''
with open("/content/llm_fa/scripts/synth_mod.ys", "w") as f:
    f.write(synth_mod_script)

!./yosys -q -s /content/llm_fa/scripts/synth_mod.ys
print("✅ Re-synthesis complete with FA_CELL instantiated.")


✅ Re-synthesis complete with FA_CELL instantiated.


In [171]:
# 11. Run synthesis before and after refactoring
original_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example_hard.v
hierarchy -auto-top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_orig.ys", "w") as f:
    f.write(original_stat_script)

mod_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -auto-top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_mod.ys", "w") as f:
    f.write(mod_stat_script)


print("\n📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'")
os.makedirs("/content/llm_fa/stats", exist_ok=True)

!./yosys -s /content/llm_fa/scripts/stat_orig.ys > /content/llm_fa/stats/stat_orig.txt
print("\n📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'")
!./yosys -s /content/llm_fa/scripts/stat_mod.ys   > /content/llm_fa/stats/stat_mod.txt



📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'

📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'


In [176]:
def print_gate_summary(stat_path):
    important_cells = ["FA_CELL", "$_AND_", "$_OR_", "$_XOR_"]
    found = {cell: 0 for cell in important_cells}

    with open(stat_path, "r") as f:
        for line in f:
            for cell in important_cells:
                if line.strip().startswith(cell):
                    count = int(line.strip().split()[-1])
                    found[cell] = count

    logic_total = 0
    for cell in important_cells:
        print(f"{cell:>8}: {found[cell]}")
        if cell.startswith("$_"):  # gate-level logic only
            logic_total += found[cell]

    print(f"{'GATES':>8}: {logic_total}")
    return logic_total
print("📊 Original design summary:")
total_orig = print_gate_summary("/content/llm_fa/stats/stat_orig.txt")

print("\n📊 Modified design summary")
total_mod = print_gate_summary("/content/llm_fa/stats/stat_mod.txt")

print(f"\n🔻 Logic gate reduction: {total_orig - total_mod} gates")

📊 Original design summary:
 FA_CELL: 0
  $_AND_: 12
   $_OR_: 8
  $_XOR_: 8
   GATES: 28

📊 Modified design summary
 FA_CELL: 4
  $_AND_: 8
   $_OR_: 4
  $_XOR_: 8
   GATES: 20

🔻 Logic gate reduction: 8 gates
